In [4]:
!pip install chromadb sentence-transformers


In [5]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer




In [6]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import tempfile

# Create a temporary directory for DuckDB+Parquet
temp_dir = tempfile.mkdtemp()

import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import tempfile

# Create a temporary directory for DuckDB+Parquet
temp_dir = tempfile.mkdtemp()

# Initialize ChromaDB client, using the temporary directory
client = chromadb.Client(Settings(
    persist_directory=temp_dir,  # Use
    anonymized_telemetry=False
))

In [7]:
# Load the Sentence Transformer model
embedder = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
# Sample data
documents = [
    {"id": "1", "text": "ChromaDB is a vector database for storing embeddings.", "category": "Tech"},
    {"id": "2", "text": "Semantic search allows finding contextually relevant content.", "category": "Tech"},
    {"id": "3", "text": "This project demonstrates a query-to-response system.", "category": "Project"},
    {"id": "4", "text": "AI applications often utilize embeddings for better accuracy.", "category": "AI"},
    {"id": "5", "text": "Vector databases are optimized for similarity search.", "category": "Tech"}
]

# Get or create a collection - replace 'my_collection' with your desired collection name
collection = client.get_or_create_collection(name="my_collection")

# Add documents to ChromaDB
for doc in documents:
    embedding = embedder.encode(doc["text"])
    collection.add(
        documents=[doc["text"]],
        metadatas=[{"category": doc["category"]}],
        ids=[doc["id"]],
        embeddings=[embedding]
    )

In [9]:
# Example query
user_query = "What are vector databases?"

# Define or import query_retriever
def query_retriever(query):
    """Retrieves results from the ChromaDB collection based on the given query."""
    results = collection.query(
        query_texts=[query],
        n_results=5  # Adjust the number of results as needed
    )
    return results


response = query_retriever(user_query)
print("Query Results:\n")
print(response)

Query Results:

{'ids': [['5', '1', '3', '4', '2']], 'embeddings': None, 'documents': [['Vector databases are optimized for similarity search.', 'ChromaDB is a vector database for storing embeddings.', 'This project demonstrates a query-to-response system.', 'AI applications often utilize embeddings for better accuracy.', 'Semantic search allows finding contextually relevant content.']], 'uris': None, 'data': None, 'metadatas': [[{'category': 'Tech'}, {'category': 'Tech'}, {'category': 'Project'}, {'category': 'AI'}, {'category': 'Tech'}]], 'distances': [[0.5773178339004517, 0.8553831577301025, 1.3801445960998535, 1.4161627292633057, 1.5133297443389893]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [10]:
!pip install gradio

In [11]:
import gradio as gr

def interactive_query(user_query):
    return query_retriever(user_query)

# Gradio Interface
interface = gr.Interface(
    fn=interactive_query,
    inputs="text",
    outputs="text",
    title="Query-to-Response System",
    description="Enter your query to retrieve the most contextually relevant content."
)

interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fbbd60fd4e26ab233e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
